In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import pyreadr
import os
from pandas.core.reshape.merge import merge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LassoCV
import seaborn as sns

In [57]:

# Read the cluster assignments
cluster_path = "/rds/general/project/hda_24-25/live/TDS/Group09/CGGM_Recoded_Cluster_Labels/cluster_assignments_k4.csv"

cluster_df = pd.read_csv(cluster_path)


In [58]:
print("Cluster data shape:", cluster_df.shape)
print("\nCluster columns:", cluster_df.columns)
print("\nCluster distribution:")
print(cluster_df['Cluster'].value_counts())

Cluster data shape: (29672, 2)

Cluster columns: Index(['index', 'Cluster'], dtype='object')

Cluster distribution:
Cluster
1    10152
2     8338
4     5705
3     5477
Name: count, dtype: int64


In [59]:
matched_data = "/rds/general/project/hda_24-25/live/TDS/Group09/Gianna/matched.csv"
matched_df = pd.read_csv(matched_data)

In [60]:
print("Exposure data shape:", matched_df.shape)
print("\nExposure columns:", matched_df.columns)

Exposure data shape: (29672, 1905)

Exposure columns: Index(['index', 'Row.names', 'sex.0.0', 'waist_circumference.0.0',
       'waist_circumference.1.0', 'waist_circumference.2.0',
       'waist_circumference.3.0', 'hip_circumference.0.0',
       'hip_circumference.1.0', 'hip_circumference.2.0',
       ...
       'WNT9A', 'WWP2', 'XCL1', 'XG', 'XPNPEP2', 'XRCC4', 'YES1', 'YTHDF3',
       'ZBTB16', 'ZBTB17'],
      dtype='object', length=1905)


In [61]:
#load composite scores for physical_activity and diet
met_score_df = pd.read_csv("/rds/general/project/hda_24-25/live/TDS/Group09/Gianna/physical_activity.csv")
diet_score_df = pd.read_csv("/rds/general/project/hda_24-25/live/TDS/Group09/Gianna/diet.csv")

In [62]:
#merge all -
merged_df = cluster_df.merge(matched_df, on='index', how='inner')
merged_df = merged_df.merge(met_score_df, on='index', how='left')
merged_df = merged_df.merge(diet_score_df, on='index', how='left')


In [63]:
print("Original shapes:")
print(f"Cluster data: {cluster_df.shape}")
print(f"Exposure data: {matched_df.shape}")
print(f"MET score data: {met_score_df.shape}")
print(f"Diet score data: {diet_score_df.shape}")
print(f"Merged data: {merged_df.shape}")

Original shapes:
Cluster data: (29672, 2)
Exposure data: (29672, 1905)
MET score data: (489213, 2)
Diet score data: (502211, 2)
Merged data: (29672, 1908)


In [64]:
(merged_df)

,index,Cluster,Row.names,sex.0.0,waist_circumference.0.0,waist_circumference.1.0,waist_circumference.2.0,waist_circumference.3.0,hip_circumference.0.0,hip_circumference.1.0,...,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17,met_score,diet_score
0,1000239,2,1000239,1,93.0,NaN,NaN,NaN,94.0,NaN,...,-0.11620,-0.60110,0.51390,-0.98185,-1.0835,-1.54115,-0.15155,0.36590,1638.0,5.5
1,1000677,2,1000677,1,93.0,NaN,NaN,NaN,99.0,NaN,...,-0.79510,-0.38350,-0.57370,-0.81665,-1.5880,-1.09355,0.00000,-0.46390,126.0,5.0
2,1000694,4,1000694,1,94.0,NaN,NaN,NaN,101.0,NaN,...,0.11690,-0.40760,-0.38480,-0.09975,1.3176,1.12845,1.10665,0.32480,0.0,5.0
3,1000886,4,1000886,0,79.0,NaN,NaN,NaN,94.0,NaN,...,0.37010,0.22510,1.02970,0.21795,2.3463,0.84285,1.05065,-0.15340,0.0,NaN
4,1001110,2,1001110,0,114.0,NaN,NaN,NaN,118.0,NaN,...,0.68020,0.46180,-1.26160,-0.53005,-2.2485,-1.14645,-0.15475,0.19590,9198.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29667,6023538,1,6023538,1,73.0,NaN,NaN,NaN,93.0,NaN,...,0.87200,-0.30495,0.76755,0.04535,0.0776,0.25065,-0.09550,-0.29075,NaN,4.5
29668,6023677,2,6023677,0,91.0,NaN,NaN,NaN,110.0,NaN,...,0.34490,0.38160,-1.43110,0.38185,-0.8904,-0.71100,0.09355,0.00180,346.5,NaN
29669,6023702,4,6023702,1,105.0,NaN,NaN,NaN,104.0,NaN,...,-0.15215,-0.49960,0.77410,0.34625,1.1100,1.50200,0.19265,0.33270,1644.0,6.5
29670,6024269,3,6024269,1,97.0,NaN,NaN,NaN,98.0,NaN,...,0.33570,0.26780,-1.11570,-0.00200,-0.1669,0.00000,-0.01975,0.03810,6132.0,5.5


In [65]:
# Check the merge results
print("Original cluster data shape:", cluster_df.shape)  # Should be (29672, 2)
print("Original exposure data shape:", matched_df.shape)  # Should be (52686, 1521)
print("Merged data shape:", merged_df.shape)
print("\nCluster distribution after merge:")
print(merged_df['Cluster'].value_counts())

Original cluster data shape: (29672, 2)
Original exposure data shape: (29672, 1905)
Merged data shape: (29672, 1908)

Cluster distribution after merge:
Cluster
1    10152
2     8338
4     5705
3     5477
Name: count, dtype: int64


In [66]:
print("Merged Columns", merged_df.columns)

Merged Columns Index(['index', 'Cluster', 'Row.names', 'sex.0.0', 'waist_circumference.0.0',
       'waist_circumference.1.0', 'waist_circumference.2.0',
       'waist_circumference.3.0', 'hip_circumference.0.0',
       'hip_circumference.1.0',
       ...
       'XCL1', 'XG', 'XPNPEP2', 'XRCC4', 'YES1', 'YTHDF3', 'ZBTB16', 'ZBTB17',
       'met_score', 'diet_score'],
      dtype='object', length=1908)


In [70]:
exposure_categories = {
    'Physical Activity': [
        'met_score',
        'diet_score',
        'number_of_daysweek_of_moderate_physical_activity_10_minutes.0.0',
        'number_of_daysweek_of_vigorous_physical_activity_10_minutes.0.0',
        'time_spent_doing_moderate_physical_activity.0.0',
        'time_spent_doing_vigorous_physical_activity.0.0'
    ],
    
    'Smoking': [
        'smoking_status.0.0',
        'current_tobacco_smoking.0.0',
        'past_tobacco_smoking.0.0',
        'smokingsmokers_in_household.0.0'
    ],
    
    'Alcohol': [
        'alcohol_intake_frequency.0.0',
        'alcohol_drinker_status.0.0'
    ],
    
    'Socioeconomic': [
        'accommodation.0.0',
        'education_score_england.0.0',
        'education_score_wales.0.0',
        'education_score_scotland.0.0',
        'index_of_multiple_deprivation_england.0.0',
        'index_of_multiple_deprivation_wales.0.0',
        'index_of_multiple_deprivation_scotland.0.0'
    ],
    
    'BMI': [
        'body_mass_index_bmi.1.0',  
    ],
    
    'Sleep': [
        'sleep_duration.0.0',
        'sleeplessness_insomnia.0.0',
    ],
    
    'Sedentary Behavior': [
        'time_spent_watching_television_tv.0.0'
    ]
}



In [71]:
#checking they are present
for category, variables in exposure_categories.items():
    print(f"\n{category}:")
    for var in variables:
        present =var in merged_df.columns
        print(f"  {var}: {'present' if present else 'missing'}")



Physical Activity:
  met_score: present
  diet_score: present
  number_of_daysweek_of_moderate_physical_activity_10_minutes.0.0: present
  number_of_daysweek_of_vigorous_physical_activity_10_minutes.0.0: present
  time_spent_doing_moderate_physical_activity.0.0: present
  time_spent_doing_vigorous_physical_activity.0.0: present

Smoking:
  smoking_status.0.0: present
  current_tobacco_smoking.0.0: present
  past_tobacco_smoking.0.0: present
  smokingsmokers_in_household.0.0: present

Alcohol:
  alcohol_intake_frequency.0.0: present
  alcohol_drinker_status.0.0: present

Socioeconomic:
  accommodation.0.0: present
  education_score_england.0.0: present
  education_score_wales.0.0: present
  education_score_scotland.0.0: present
  index_of_multiple_deprivation_england.0.0: present
  index_of_multiple_deprivation_wales.0.0: present
  index_of_multiple_deprivation_scotland.0.0: present

BMI:
  body_mass_index_bmi.1.0: present

Sleep:
  sleep_duration.0.0: present
  sleeplessness_insomnia.

In [72]:
# Function to create Table 1
def create_table1(df, categories):
    results = []
    
    for category, variables in categories.items():
        for var in variables:
            row = {'Variable': var, 'Category': category}
            
            # Calculate statistics for each cluster
            for cluster in sorted(df['Cluster'].unique()):
                cluster_data = df[df['Cluster'] == cluster][var]
                
                # Check if numeric
                if pd.api.types.is_numeric_dtype(cluster_data):
                    # For numeric variables: mean ± SD
                    mean = cluster_data.mean()
                    std = cluster_data.std()
                    row[f'Cluster {cluster}'] = f"{mean:.2f} ± {std:.2f}"
                    
                    # ANOVA for p-value
                    groups = [group[var].dropna() for name, group in df.groupby('Cluster')]
                    try:
                        f_stat, p_val = stats.f_oneway(*groups)
                    except:
                        p_val = np.nan
                else:
                    # For categorical variables: percentages
                    value_counts = cluster_data.value_counts(normalize=True) * 100
                    row[f'Cluster {cluster}'] = ', '.join([f"{k}: {v:.1f}%" for k, v in value_counts.items()])
                    
                    # Chi-square for p-value
                    try:
                        contingency = pd.crosstab(df['Cluster'], df[var])
                        chi2, p_val = stats.chi2_contingency(contingency)[:2]
                    except:
                        p_val = np.nan
            
            row['p-value'] = f"{p_val:.4f}" if not np.isnan(p_val) else 'NA'
            results.append(row)
    
    return pd.DataFrame(results)



In [73]:
# Create Table 1
table1 = create_table1(merged_df, exposure_categories)


In [74]:
# Display Table 1
print("\nTable 1:")
print(table1.to_string())

# Save Table 1 to CSV if needed
table1.to_csv('table1_cluster_exposures.csv', index=False)


Table 1:
                                                           Variable            Category          Cluster 1          Cluster 2          Cluster 3          Cluster 4 p-value
0                                                         met_score   Physical Activity  2430.71 ± 2656.46  2460.01 ± 2658.75  2094.48 ± 2494.67  2307.35 ± 2605.31  0.0000
1                                                        diet_score   Physical Activity        5.87 ± 1.64        5.94 ± 1.65        5.62 ± 1.72        5.65 ± 1.73  0.0000
2   number_of_daysweek_of_moderate_physical_activity_10_minutes.0.0   Physical Activity        3.44 ± 2.51        3.43 ± 2.53        3.01 ± 2.65        3.27 ± 2.58  0.0000
3   number_of_daysweek_of_vigorous_physical_activity_10_minutes.0.0   Physical Activity        1.70 ± 2.05        1.71 ± 2.09        1.24 ± 2.03        1.52 ± 2.07  0.0000
4                   time_spent_doing_moderate_physical_activity.0.0   Physical Activity   678.84 ± 1035.33   663.03 ± 1051.44   66

In [75]:
# Function to create Table 1 with improved formatting
def create_table1(df, categories):
    results = []
    
    for category, variables in categories.items():
        # Add category header
        results.append({
            'Variable': f"\n{category}",
            'Category': '',
            'Cluster 1': '',
            'Cluster 2': '',
            'Cluster 3': '',
            'Cluster 4': '',
            'p-value': ''
        })
        
        for var in variables:
            row = {'Variable': var, 'Category': category}
            
            # Calculate statistics for each cluster
            for cluster in sorted(df['Cluster'].unique()):
                cluster_data = df[df['Cluster'] == cluster][var]
                
                if pd.api.types.is_numeric_dtype(cluster_data):
                    # For numeric variables: mean ± SD
                    mean = cluster_data.mean()
                    std = cluster_data.std()
                    n = cluster_data.count()  # number of non-null values
                    row[f'Cluster {cluster}'] = f"{mean:.1f} ± {std:.1f} (n={n})"
                    
                    # ANOVA for p-value
                    groups = [group[var].dropna() for name, group in df.groupby('Cluster')]
                    try:
                        f_stat, p_val = stats.f_oneway(*groups)
                    except:
                        p_val = np.nan
                else:
                    # For categorical variables: n (%)
                    value_counts = cluster_data.value_counts()
                    total = value_counts.sum()
                    percentages = (value_counts / total * 100).round(1)
                    row[f'Cluster {cluster}'] = ', '.join([f"{k}: {v}n ({p}%)" 
                                                         for k, v, p in zip(value_counts.index, 
                                                                          value_counts.values, 
                                                                          percentages)])
                    
                    # Chi-square for p-value
                    try:
                        contingency = pd.crosstab(df['Cluster'], df[var])
                        chi2, p_val = stats.chi2_contingency(contingency)[:2]
                    except:
                        p_val = np.nan
            
            # Format p-value with stars for significance
            if not np.isnan(p_val):
                stars = ''
                if p_val < 0.001:
                    stars = '***'
                elif p_val < 0.01:
                    stars = '**'
                elif p_val < 0.05:
                    stars = '*'
                row['p-value'] = f"{p_val:.3f}{stars}"
            else:
                row['p-value'] = 'NA'
            
            results.append(row)
    
    table = pd.DataFrame(results)
    
    # Add total N row at the top
    total_n = {
        'Variable': 'Total N',
        'Category': '',
    }
    for cluster in sorted(df['Cluster'].unique()):
        total_n[f'Cluster {cluster}'] = str(len(df[df['Cluster'] == cluster]))
    total_n['p-value'] = ''
    
    table = pd.concat([pd.DataFrame([total_n]), table], ignore_index=True)
    
    return table



In [76]:
# Create and display Table 1
table1 = create_table1(merged_df, exposure_categories)


In [78]:


# Save Table 1 to CSV
table1.to_csv('table1_cluster_exposures.csv', index=False)


In [77]:


# Display first few rows to check formatting
print(table1.head(10))

                                            Variable           Category  \
0                                            Total N                      
1                                \nPhysical Activity                      
2                                          met_score  Physical Activity   
3                                         diet_score  Physical Activity   
4  number_of_daysweek_of_moderate_physical_activi...  Physical Activity   
5  number_of_daysweek_of_vigorous_physical_activi...  Physical Activity   
6    time_spent_doing_moderate_physical_activity.0.0  Physical Activity   
7    time_spent_doing_vigorous_physical_activity.0.0  Physical Activity   
8                                          \nSmoking                      
9                                 smoking_status.0.0            Smoking   

                  Cluster 1                 Cluster 2  \
0                     10152                      8338   
1                                                       
2  

From here on is LASSO

In [80]:

# Define the exposure variables (using the same ones from exposure_categories)
exposure_vars = (
    # Physical Activity
    ['met_score', 'diet_score',
     'number_of_daysweek_of_moderate_physical_activity_10_minutes.0.0',
     'number_of_daysweek_of_vigorous_physical_activity_10_minutes.0.0',
     'time_spent_doing_moderate_physical_activity.0.0',
     'time_spent_doing_vigorous_physical_activity.0.0',
    
    # Smoking
     'smoking_status.0.0', 'current_tobacco_smoking.0.0',
     'past_tobacco_smoking.0.0', 'smokingsmokers_in_household.0.0',
    
    # Alcohol
     'alcohol_intake_frequency.0.0', 'alcohol_drinker_status.0.0',
    
    # Socioeconomic
     'accommodation.0.0', 'education_score_england.0.0',
     'education_score_wales.0.0', 'education_score_scotland.0.0',
     'index_of_multiple_deprivation_england.0.0',
     'index_of_multiple_deprivation_wales.0.0',
     'index_of_multiple_deprivation_scotland.0.0',
    
    # BMI
     'body_mass_index_bmi.1.0',
    
    # Sleep
     'sleep_duration.0.0', 'sleeplessness_insomnia.0.0',
    
    # Sedentary Behavior
     'time_spent_watching_television_tv.0.0']
)


In [81]:

def stability_lasso(X, y, n_iterations=100, subset_fraction=0.8):
    n_samples, n_features = X.shape
    feature_selection_freq = np.zeros((n_iterations, n_features))
    
    for i in range(n_iterations):
        # Random subsample
        indices = np.random.choice(n_samples, size=int(n_samples * subset_fraction), replace=False)
        X_subset = X[indices]
        y_subset = y[indices]
        
        # Fit LASSO
        lasso = LassoCV(cv=5, random_state=i)
        lasso.fit(X_subset, y_subset)
        
        # Record selected features
        feature_selection_freq[i, :] = np.abs(lasso.coef_) > 0
    
    # Calculate selection frequency for each feature
    selection_frequency = np.mean(feature_selection_freq, axis=0)
    
    # Create results DataFrame
    results = pd.DataFrame({
        'Feature': exposure_vars,
        'Selection_Frequency': selection_frequency
    }).sort_values('Selection_Frequency', ascending=False)
    
    return results


In [86]:

# Prepare data for LASSO
# First, handle missing values
X = merged_df[exposure_vars].copy()
X = X.fillna(X.median())


In [87]:

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [88]:

# Prepare target variable (cluster labels)
y = merged_df['Cluster'].values


In [ ]:
# Run stability LASSO
np.random.seed(42)  
lasso_results = stability_lasso(X_scaled, y)

In [ ]:

# Print results
print("Stability LASSO Results:")
print(lasso_results)



In [ ]:
# Create plot
plt.figure(figsize=(12, 8))
sns.barplot(data=lasso_results, x='Selection_Frequency', y='Feature')
plt.title('Feature Selection Frequency in Stability LASSO')
plt.xlabel('Selection Frequency')
plt.ylabel('Features')
plt.tight_layout()
plt.savefig('stability_lasso_plot.pdf')
plt.show()



In [ ]:

# Print top 10 most important features
print("\nTop 10 Most Important Features:")
print(lasso_results.head(10))

In [ ]:
# Save results
lasso_results.to_csv('stability_lasso_results.csv', index=False)


In [6]:
table1csv= pd.read_csv("table1_cluster_exposures.csv")
(table1csv)

,Variable,Category,Cluster 1,Cluster 2,Cluster 3,Cluster 4,p-value
0,Total N,NaN,10152,8338,5477,5705,NaN
1,\nPhysical Activity,NaN,NaN,NaN,NaN,NaN,NaN
2,met_score,Physical Activity,2430.7 ± 2656.5 (n=9712),2460.0 ± 2658.8 (n=7947),2094.5 ± 2494.7 (n=5063),2307.4 ± 2605.3 (n=5390),0.000***
3,diet_score,Physical Activity,5.9 ± 1.6 (n=8382),5.9 ± 1.7 (n=6863),5.6 ± 1.7 (n=4476),5.7 ± 1.7 (n=4669),0.000***
4,number_of_daysweek_of_moderate_physical_activi...,Physical Activity,3.4 ± 2.5 (n=10137),3.4 ± 2.5 (n=8332),3.0 ± 2.7 (n=5475),3.3 ± 2.6 (n=5698),0.000***
5,number_of_daysweek_of_vigorous_physical_activi...,Physical Activity,1.7 ± 2.1 (n=10137),1.7 ± 2.1 (n=8332),1.2 ± 2.0 (n=5475),1.5 ± 2.1 (n=5698),0.000***
6,time_spent_doing_moderate_physical_activity.0.0,Physical Activity,678.8 ± 1035.3 (n=1526),663.0 ± 1051.4 (n=917),665.0 ± 1052.6 (n=651),721.8 ± 1100.7 (n=900),0.625
7,time_spent_doing_vigorous_physical_activity.0.0,Physical Activity,385.3 ± 875.6 (n=1526),380.3 ± 875.4 (n=917),367.9 ± 853.0 (n=651),405.6 ± 876.8 (n=900),0.855
8,\nSmoking,NaN,NaN,NaN,NaN,NaN,NaN
9,smoking_status.0.0,Smoking,0.5 ± 0.7 (n=10137),0.5 ± 0.7 (n=8332),0.6 ± 0.7 (n=5475),0.6 ± 0.7 (n=5698),0.000***
